<a href="https://colab.research.google.com/github/lendercity/tqqq-signals/blob/main/TQQQ_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# TQQQ Daily Trading Signal with Probabilistic Position Sizing

import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import joblib
import os

MODEL_FILE = "tqqq_model.pkl"
SCALER_FILE = "tqqq_scaler.pkl"

# ===== 1. Download data =====
ticker = "TQQQ"
data = yf.download(ticker, period="5y", interval="1d", auto_adjust=False)
data['Return'] = data['Close'].pct_change()

# Ensure Close is Series
close = data['Close']
if isinstance(close, pd.DataFrame):
    close = close.iloc[:, 0]
data['Close_Only'] = close

# ===== 2. Features =====
data['SMA10'] = data['Close_Only'].rolling(10).mean()
data['SMA20'] = data['Close_Only'].rolling(20).mean()
data['EMA10'] = data['Close_Only'].ewm(span=10, adjust=False).mean()
data['EMA20'] = data['Close_Only'].ewm(span=20, adjust=False).mean()
data['Momentum'] = data['Close_Only'] - data['Close_Only'].shift(5)
data['Volatility'] = data['Close_Only'].rolling(10).std()

# MACD
data['EMA12'] = data['Close_Only'].ewm(span=12, adjust=False).mean()
data['EMA26'] = data['Close_Only'].ewm(span=26, adjust=False).mean()
data['MACD'] = data['EMA12'] - data['EMA26']
data['Signal_line'] = data['MACD'].ewm(span=9, adjust=False).mean()

# Bollinger Bands
window = 20
bb_mid = data['Close_Only'].rolling(window, min_periods=window).mean()
bb_std = data['Close_Only'].rolling(window, min_periods=window).std()
data['BB_Mid'] = bb_mid
data['BB_Upper'] = bb_mid + 2 * bb_std
data['BB_Lower'] = bb_mid - 2 * bb_std

# RSI
delta = data['Close_Only'].diff()
gain = np.where(delta > 0, delta, 0)
loss = np.where(delta < 0, -delta, 0)
avg_gain = pd.Series(gain, index=data.index).rolling(14).mean()
avg_loss = pd.Series(loss, index=data.index).rolling(14).mean()
rs = avg_gain / avg_loss
data['RSI'] = 100 - (100 / (1 + rs))

# Target
data['Target'] = np.where(data['Return'].shift(-1) > 0, 1, 0)
data.dropna(inplace=True)

features = [
    'SMA10','SMA20','EMA10','EMA20','Momentum',
    'Volatility','MACD','Signal_line','BB_Upper','BB_Lower','RSI'
]
X = data[features]
y = data['Target']

# ===== 3. Train/Test Split =====
X_train = X.iloc[:-1]
y_train = y.iloc[:-1]

# Load or fit scaler
if os.path.exists(SCALER_FILE):
    scaler = joblib.load(SCALER_FILE)
else:
    scaler = StandardScaler()
    scaler.fit(X_train)
    joblib.dump(scaler, SCALER_FILE)

X_train_scaled = scaler.transform(X_train)

# Load or train model
if os.path.exists(MODEL_FILE):
    model = joblib.load(MODEL_FILE)
else:
    model = RandomForestClassifier(n_estimators=300, max_depth=5, random_state=42)
    model.fit(X_train_scaled, y_train)
    joblib.dump(model, MODEL_FILE)

# ===== 4. Today's Signal =====
today_features = X.iloc[-1].values.reshape(1, -1)
today_scaled = scaler.transform(today_features)

signal = model.predict(today_scaled)[0]
proba = model.predict_proba(today_scaled)[0][1]  # probability of UP (class 1)

# Map probability -> position size (10% to 100%)
if signal == 1:
    # position grows with confidence above 0.5
    position_size = max(0.1, min(1.0, proba))
    signal_text = f"TQQQ Signal: BUY | Confidence: {proba:.2f} | Position: {position_size*100:.0f}%"
else:
    # SELL/short signal → confidence is probability of DOWN = 1 - proba
    confidence = 1 - proba
    position_size = max(0.1, min(1.0, confidence))
    signal_text = f"TQQQ Signal: SELL | Confidence: {confidence:.2f} | Position: {position_size*100:.0f}%"

print(signal_text)


[*********************100%***********************]  1 of 1 completed

TQQQ Signal: BUY | Confidence: 0.55 | Position: 55%
